In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(10,))

# Declare layers
layer1 = Dense(64, activation="relu")
layer2 = Dense(64, activation="relu")

# Connect inputs and layers
layer1_outputs = layer1(inputs)
layer2_outputs = layer2(layer1_outputs)

# Create model.
model = Model(inputs=inputs, outputs=layer2_outputs)
model.summary()

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate

# Declare inputs
inputs = Input(shape=(10,))
bypass_inputs = Input(shape=(5,))

# Declaration of layers
layer1 = Dense(64, activation="relu")
concat_layer = Concatenate()
layer2 = Dense(64, activation="relu")

# Conection
layers1_output = layer1(inputs)
layers2_inputs = concat_layer([layer1_outputs, bypass_inputs])
layer2_outputs = layer2(layers2_inputs)

# Create model
model = Model(inputs=[inputs, bypass_inputs], outputs=layer2_outputs)
model.summary()

In [3]:
import numpy as np
import random
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import logging

tf.get_logger().setLevel(logging.ERROR)

In [4]:
EPOCHS = 20
BATCH_SIZE = 128
MAX_WORDS = 10000
READ_LINES = 60000
LAYER_SIZE = 256
EMBEDDING_WIDTH = 128
TEST_PERCENT = 0.2
SAMPLE_SIZE = 20
OOV_WORD = "UNK"
PAD_INDEX = 0
OOV_INDEX = 1
START_INDEX = MAX_WORDS - 2
STOP_INDEX = MAX_WORDS - 1
MAX_LENGTH = 60
SRC_DEST_FILE_NAME = "../data/fra-eng/fra.txt"

In [ ]:
def read_file_combined(file_name, max_len):
    file = open(file_name, "r", encoding="utf-8")
    src_word_sequences = []
    dest_word_sequences = []
    for i, line in enumerate(file):
        if i == READ_LINES:
            break
        pair = line.split("\t")
        word_sequence = text_to_word_sequence(pair[1])
        src_word_sequence = word_sequence[0:max_len]
        src_word_sequences.append(src_word_sequence)
        word_sequence = text_to_word_sequence(pair[0])
        dest_word_sequence = word_sequence[0:max_len]
        dest_word_sequences.append(dest_word_sequence)
    file.close()
    return src_word_sequences, dest_word_sequences


def tokenize(sequences):

    tokenizer = Tokenizer(num_words=MAX_WORDS - 2, oov_token=OOV_WORD)
    tokenizer.fit_on_texts(sequences)
    token_sequences = tokenizer.texts_to_sequences(sequences)
    return tokenizer, token_sequences


def tokens_to_words(tokenizer, seq):
    word_seq = []
    for index in seq:
        if index == PAD_INDEX:
            word_seq.append("PAD")
        elif index == OOV_INDEX:
            word_seq.append(OOV_WORD)
        elif index == START_INDEX:
            word_seq.append("START")
        elif index == STOP_INDEX:
            word_seq.append("STOP")
        else:
            word_seq.append(tokenizer.sequences_to_texts([[index]])[0])
    print(word_seq)

In [6]:
src_seq, dest_seq = read_file_combined(SRC_DEST_FILE_NAME, MAX_LENGTH)
src_tokenizer, src_token_seq = tokenize(src_seq)
dest_tokenizer, dest_token_seq = tokenize(dest_seq)

In [ ]:
dest_target_token_seq = [x + [STOP_INDEX] for x in dest_token_seq]
dest_input_token_seq = [[START_INDEX] + x for x in dest_target_token_seq]
src_input_data = pad_sequences(src_token_seq)
dest_input_data = pad_sequences(dest_input_token_seq, padding="post")
dest_target_data = pad_sequences(
    dest_target_token_seq, padding="post", maxlen=len(dest_input_data[0])
)

In [ ]:
rows = len(src_input_data[:, 0])
all_indices = list(range(rows))
test_rows = int(rows * TEST_PERCENT)
test_indices = random.sample(all_indices, test_rows)
train_indices = [x for x in all_indices if x not in test_indices]

train_src_input_data = src_input_data[train_indices]
train_dest_input_data = dest_input_data[train_indices]
train_dest_target_data = dest_target_data[train_indices]

test_src_input_data = src_input_data[test_indices]
test_dest_input_data = dest_input_data[test_indices]
test_dest_target_data = dest_target_data[test_indices]

# 20 samples
test_indices = list(range(test_rows))
sample_indices = random.sample(test_indices, SAMPLE_SIZE)
sample_input_data = test_src_input_data[sample_indices]
sample_target_data = test_dest_target_data[sample_indices]

In [ ]:
# Encoder model
enc_embedding_input = Input(shape=(None,))

enc_embedding_layer = Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS, mask_zero=True
)

enc_layer1 = LSTM(LAYER_SIZE, return_state=True, return_sequences=True)

enc_layer2 = LSTM(LAYER_SIZE, return_state=True)

enc_embedding_layer_outputs = enc_embedding_layer(enc_embedding_input)

enc_layer1_outputs, enc_layer1_state_h, enc_layer1_state_c = enc_layer1(
    enc_embedding_layer_outputs
)

_, enc_layer2_state_h, enc_layer2_state_c = enc_layer2(enc_layer1_outputs)

enc_model = Model(
    enc_embedding_input,
    [enc_layer1_state_h, enc_layer1_state_c, enc_layer2_state_h, enc_layer2_state_c],
)

enc_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 128) │  1,280,000 │ input_layer_6[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_2         │ (None, None)      │          0 │ input_layer_6[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, None,     │    394,240 │ embedding_2[0][0… │
│                     │ 256), (None,      │            │ not_equal_2[0][0] │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 256),     │    525,312 │ lstm_4[0][0],     │
│                     │ (None, 256),      │            │ not_equal_2[0][0] │
│                     │ (None, 256)]      │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,199,552 (8.39 MB)

 Trainable params: 2,199,552 (8.39 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Decoder model

dec_layer1_state_input_h = Input(shape=(LAYER_SIZE,))
dec_layer1_state_input_c = Input(shape=(LAYER_SIZE,))

dec_layer2_state_input_h = Input(shape=(LAYER_SIZE,))
dec_layer2_state_input_c = Input(shape=(LAYER_SIZE,))
dec_embedding_input = Input(shape=(None,))

# Create the decoder layers

dec_embedding_layer = Embedding(
    output_dim=EMBEDDING_WIDTH, input_dim=MAX_WORDS, mask_zero=True
)
dec_layer1 = LSTM(LAYER_SIZE, return_state=True, return_sequences=True)
dec_layer2 = LSTM(LAYER_SIZE, return_state=True, return_sequences=True)
dec_layer3 = Dense(MAX_WORDS, activation="softmax")

# Connect the decoder layers

dec_embedding_layer_outputs = dec_embedding_layer(dec_embedding_input)

dec_layer1_outputs, dec_layer1_state_h, dec_layer1_state_c = dec_layer1(
    dec_embedding_layer_outputs,
    initial_state=[dec_layer1_state_input_h, dec_layer1_state_input_c],
)

dec_layer2_outputs, dec_layer2_state_h, dec_layer2_state_c = dec_layer2(
    dec_layer1_outputs,
    initial_state=[dec_layer2_state_input_h, dec_layer2_state_input_c],
)

dec_layer3_outputs = dec_layer3(dec_layer2_outputs)

dec_model = Model(
    [
        dec_embedding_input,
        dec_layer1_state_input_h,
        dec_layer1_state_input_c,
        dec_layer2_state_input_h,
        dec_layer2_state_input_c,
    ],
    [
        dec_layer3_outputs,
        dec_layer1_state_h,
        dec_layer1_state_c,
        dec_layer2_state_h,
        dec_layer2_state_c,
    ],
)

dec_model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_16      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, None, 128) │  1,280,000 │ input_layer_16[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_12      │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_13      │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_8 (LSTM)       │ [(None, None,     │    394,240 │ embedding_4[0][0… │
│                     │ 256), (None,      │            │ input_layer_12[0… │
│                     │ 256), (None,      │            │ input_layer_13[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_14      │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, 256)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_9 (LSTM)       │ [(None, None,     │    525,312 │ lstm_8[0][0],     │
│                     │ 256), (None,      │            │ input_layer_14[0… │
│                     │ 256), (None,      │            │ input_layer_15[0… │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, None,      │  2,570,000 │ lstm_9[0][0]      │
│                     │ 10000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,769,552 (18.19 MB)

 Trainable params: 4,769,552 (18.19 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
train_enc_embedding_input = Input(shape=(None,))
train_dec_embedding_input = Input(shape=(None,))
intermediate_state = enc_model(train_enc_embedding_input)
train_dec_output, _, _, _, _ = dec_model(
    [train_dec_embedding_input] + intermediate_state
)
training_model = Model(
    [train_enc_embedding_input, train_dec_embedding_input], train_dec_output
)
optimizer = RMSprop(learning_rate=0.01)
training_model.compile(
    loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
)
training_model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_23      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ [(None, 256),     │  2,199,552 │ input_layer_22[0… │
│ (Functional)        │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_4        │ [(None, None,     │  4,769,552 │ input_layer_23[0… │
│ (Functional)        │ 10000), (None,    │            │ functional_2[2][… │
│                     │ 256), (None,      │            │ functional_2[2][… │
│                     │ 256), (None,      │            │ functional_2[2][… │
│                     │ 256), (None,      │            │ functional_2[2][… │
│                     │ 256)]             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,969,104 (26.59 MB)

 Trainable params: 6,969,104 (26.59 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
for i in range(EPOCHS):
    print("step: ", i)
    history = training_model.fit(
        [train_src_input_data, train_dest_input_data],
        train_dest_target_data,
        validation_data=(
            [test_src_input_data, test_dest_input_data],
            test_dest_target_data,
        ),
        batch_size=BATCH_SIZE,
        epochs=1,
    )
    for (test_input,test_target) in zip(sample_input_data, sample_target_data):
        x = np.reshape(test_input,(1,-1))
        last_states = enc_model.predict(x,verbose=0)

        prev_word_index = START_INDEX
        produced_string = ''
        pred_seq = []
        for j in range(MAX_LENGTH):
            x = np.reshape(np.array(prev_word_index),(1,1))
            preds, dec_layer1_state_h, dec_layer1_state_c, dec_layer2_state_h, dec_layer2_state_c = dec_model.predict([x] + last_states ,verbose=0)
            last_states = [dec_layer1_state_h,dec_layer1_state_c,dec_layer2_state_h,dec_layer2_state_c]
            prev_word_index = np.asarray(preds[0][0]).argmax()
            pred_seq.append(prev_word_index)

            if prev_word_index == STOP_INDEX:
                break
        tokens_to_words(src_tokenizer,test_input)
        tokens_to_words(dest_tokenizer,test_target)
        tokens_to_words(dest_tokenizer,pred_seq)
        print('\n\n')

step:  0
375/375 ━━━━━━━━━━━━━━━━━━━━ 271s 724ms/step - accuracy: 0.7382 - loss: 0.1200 - val_accuracy: 0.6412 - val_loss: 1.2721
['PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', "j'ai", 'couru', 'à', 'la', 'vitesse', 'de', 'UNK']
['i', 'ran', 'like', 'lightning', 'STOP', 'PAD', 'PAD', 'PAD', 'PAD']
['i', 'ran', 'back', 'in', 'a', 'lake', 'STOP']



['PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'signez', 'juste', 'ceci']
['just', 'sign', 'this', 'STOP', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
["let's", 'sign', 'this', 'STOP']



['PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'ça', "n'était", 'pas', 'drôle']
['it', "wasn't", 'fun', 'STOP', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']
['that', "wasn't", 'funny', 'STOP']



['PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'vous', "n'y", 'avez', 'pas', 'brillé']
['you', 'were', 'bad', 'at', 'it', 'STOP', 'PAD', 'PAD', 'PAD']
['you', 'were', 'bad', 'at', 'it', 'STOP']



['PAD', 'PAD'